In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
from icecream import ic
from matplotlib import pyplot as plt
import sys
sys.path.append('../../')

%load_ext autoreload
%autoreload 2

LOCALS = ['NVD4K9M6K4', 'x1']
import socket
hostname = socket.gethostname()
IS_LOCAL = hostname in LOCALS
def view_df(data: pd.DataFrame):
    pd.set_option('display.max_columns', None)
    display(data)
    pd.reset_option("max_columns")

impact_path = Path('./data/')
processed_data_path = impact_path / 'processed-data.csv'
label_data_path = impact_path / 'label-data.csv'

# Load data & Preprocessing

In [2]:
import random
# nrows= 100000 if IS_LOCAL else None
# n = 19433497-1
n = 19433497
s = 1000000
# IS_LOCAL = False

skip = sorted(random.sample(range(1, n+1), n-s)) if IS_LOCAL else None
input = pd.read_csv(
    processed_data_path, index_col='index',
    dtype={
        'protocol': 'string',
        'rootId': 'string',
        # 'source.port': str,
        # 'destination.port': str,
    },
    skiprows=skip if IS_LOCAL else None,
    # nrows=s,
)
label = pd.read_csv(
    label_data_path, 
    index_col='index',
    # dtype={
    #     'is_multicast': bool,
    #     'is_private': bool,
    #     'is_global': bool,
    #     'is_unspecified': bool,
    #     'is_reserved': bool,
    #     'is_loopback': bool,
    #     'is_link_local': bool
    # },
    skiprows=skip if IS_LOCAL else None,
    # nrows=s,
)

# data = pd.concat([input.drop(['source.ip'], axis=1), label], axis=1)
# data = input.dr
label = label.drop(['source.ip'], axis=1)

# if IS_LOCAL:
#     data = data.sample(10000)

/Users/q.nguyen/opt/anaconda3/envs/graphic/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3460: DtypeWarning: Columns (21,26) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/q.nguyen/opt/anaconda3/envs/graphic/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3460: DtypeWarning: Columns (19) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
# # convert column that its values are array as string type to native python array
import math

data = input.copy()

def to_python_array(val: str):
    if not val or (isinstance(val, float) and math.isnan(val)): 
        return []
    assert val[0] == '[' and val[-1] == ']', f"Failed: {val}"
    val = val[1:-1]
    return val.split(' ')


array_str_cols = ['protocol', 'dns.host']
label_array_str_cols = ['service', 'critservice', 'dns_lookup_alias']
# for col in array_str_cols:
#     # data[col] = data[col].apply(to_python_array)


In [4]:

timestamp_columns = ['firstPacket', 'lastPacket']
for col in timestamp_columns:
    data[f'{col}'] = pd.to_datetime(data[col], unit='ms')

start_time = data['firstPacket'].min()
end_time = data['lastPacket'].max()
data['time_idx'] = ((data['lastPacket'] - start_time).dt.ceil('ms').dt.total_seconds()*1000).astype(np.int64)
data['time_from_start_to_last_packet'] = (data['lastPacket'] - start_time).dt.total_seconds()
data['time_from_start_to_first_packet'] = (data['firstPacket'] - start_time).dt.total_seconds()


# g = data.groupby(['destination.ip', 'destination.port'], as_index=False)

# def add_groupwise_time_idx(in_: pd.DataFrame):
#     idx = pd.DataFrame(data={'g_time_idx': list(range(in_.shape[0]))}, dtype=np.int64)

#     out = pd.concat([
#         in_.reset_index(drop=True).sort_values(by='time_idx'), 
#         idx.reset_index(drop=True)
#     ], axis=1)
#     return out

# data = g.apply(add_groupwise_time_idx)


data['hour'] = data['lastPacket'].dt.hour
# data['day'] = data['lastPacket'].day
# data['day_of_week'] = data['lastPacket'].dayofweek
data['minute'] = data['lastPacket'].dt.minute
data['second'] = data['lastPacket'].dt.second
data['microsecond'] = data['lastPacket'].dt.microsecond

CYCLIC_COLS = [
    'hour',
    'minute',
    'second',
    'microsecond'
]

for col in CYCLIC_COLS:
    data[f"{col}_sin"]  = np.sin((2*np.pi*data[col])/max(data[col]))
    data[f"{col}_cos"]  = np.cos((2*np.pi*data[col])/max(data[col]))


data['has_cert'] = data['cert'].notna()
data['has_dns_host'] = data['dns.host'].notna()


# data['destination.port.s'] = data['destination.port'].astype(str)

# data.rename(
#     columns={
#         name: name.replace('.', '_') for name in list(data.columns)
#     },
#     inplace=True
# )
# data.replace('', np.nan)

# testing_cutoff = pd.to_datetime('2016-04-22 21:00:00')
# testing_df = data[data['lastPacket'] > testing_cutoff]


# validation_cutoff = pd.to_datetime('2016-04-22 20:30:00')
# validation_df = data[(data['lastPacket'] > validation_cutoff) & (data['lastPacket'] < testing_cutoff)]

# training_df = data[data['lastPacket'] <= validation_cutoff]
if IS_LOCAL:
    view_df(data)
display(data.info())
_ = display(label.info())

,firstPacket,lastPacket,ipProtocol,protocol,rootId,totDataBytes,client.bytes,server.bytes,source.ip,source.port,destination.ip,destination.port,network.packets,source.packets,destination.packets,network.bytes,source.bytes,destination.bytes,node,http.uri,source.geo.country_iso_code,destination.geo.country_iso_code,dns.host,cert,irc.channel,http.xffGEO,time_idx,time_from_start_to_last_packet,time_from_start_to_first_packet,hour,minute,second,microsecond,hour_sin,hour_cos,minute_sin,minute_cos,second_sin,second_cos,microsecond_sin,microsecond_cos,has_cert,has_dns_host
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,2016-04-22 14:00:00.006,2016-04-22 14:24:00.122,6,[tcp],160422-3gNaohIY_a9CmIgI7Hkjn6eF,0,0,0,192.168.223.11,902,192.168.223.177,49858,4685,2295,2390,1885844,1610152,275692,cma-arkime,NaN,NaN,NaN,[],NaN,NaN,NaN,1440116,1440.116,0.000,14,24,0,122000,-6.310879e-01,-0.775711,0.552800,-0.833314,0.000000,1.000000,0.694206,0.719777,False,True
3,2016-04-22 14:00:00.006,2016-04-22 14:40:00.122,6,[tcp],160422-3gNaohIY_a9CmIgI7Hkjn6eF,0,0,0,192.168.223.11,902,192.168.223.177,49858,86,39,47,14071,8640,5431,cma-arkime,NaN,NaN,NaN,[],NaN,NaN,NaN,2400116,2400.116,0.000,14,40,0,122000,-6.310879e-01,-0.775711,-0.899312,-0.437307,0.000000,1.000000,0.694206,0.719777,False,True
24,2016-04-22 14:00:00.062,2016-04-22 14:02:02.720,6,[tcp],160422-3gMBN_bwBMBOIaYvGs-w6P3R,0,0,0,91.189.88.148,80,10.222.83.2,50340,10000,5860,4140,9165504,8872040,293464,cma-arkime,NaN,GB,NaN,[],NaN,NaN,NaN,122714,122.714,0.056,14,2,2,720000,-6.310879e-01,-0.775711,0.211383,0.977403,0.211383,0.977403,-0.983126,-0.182931,False,True
30,2016-04-22 14:00:00.062,2016-04-22 14:04:17.049,6,[tcp],160422-3gMBN_bwBMBOIaYvGs-w6P3R,0,0,0,91.189.88.148,80,10.222.83.2,50340,10000,5792,4208,9073848,8769088,304760,cma-arkime,NaN,GB,NaN,[],NaN,NaN,NaN,257043,257.043,0.056,14,4,17,49000,-6.310879e-01,-0.775711,0.413212,0.910635,0.971430,-0.237327,0.303329,0.952886,False,True
49,2016-04-22 14:00:00.062,2016-04-22 14:12:34.574,6,[tcp],160422-3gMBN_bwBMBOIaYvGs-w6P3R,0,0,0,91.189.88.148,80,10.222.83.2,50340,10000,5819,4181,9110516,8809966,300550,cma-arkime,NaN,GB,NaN,[],NaN,NaN,NaN,754568,754.568,0.056,14,12,34,574000,-6.310879e-01,-0.775711,0.957422,0.288692,-0.461093,-0.887352,-0.451607,-0.892217,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19433444,2016-04-22 23:28:25.445,2016-04-22 23:28:25.445,6,[tcp],<NA>,0,0,0,10.180.108.77,61115,10.10.10.205,9220,1,1,0,60,60,0,cma-arkime,NaN,NaN,NaN,[],NaN,NaN,NaN,34105439,34105.439,34105.439,23,28,25,445000,-2.449294e-16,1.000000,0.159063,-0.987268,0.461093,-0.887352,0.336103,-0.941825,False,True
19433454,2016-04-22 23:28:25.459,2016-04-22 23:28:25.459,6,[tcp],<NA>,0,0,0,10.180.108.77,59578,172.16.50.205,16018,1,1,0,60,60,0,cma-arkime,NaN,NaN,NaN,[],NaN,NaN,NaN,34105453,34105.453,34105.453,23,28,25,459000,-2.449294e-16,1.000000,0.159063,-0.987268,0.461093,-0.887352,0.251978,-0.967733,False,True
19433458,2016-04-22 23:28:25.474,2016-04-22 23:28:25.477,6,[http tcp],<NA>,483,367,116,172.16.20.203,57823,10.200.97.3,80,11,6,5,1163,727,436,cma-arkime,['10.200.97.3/j.ad'],NaN,NaN,[],NaN,NaN,NaN,34105471,34105.471,34105.468,23,28,25,477000,-2.449294e-16,1.000000,0.159063,-0.987268,0.461093,-0.887352,0.141041,-0.990004,False,True


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000000 entries, 1 to 19433495
Data columns (total 43 columns):
 #   Column                            Non-Null Count    Dtype         
---  ------                            --------------    -----         
 0   firstPacket                       1000000 non-null  datetime64[ns]
 1   lastPacket                        1000000 non-null  datetime64[ns]
 2   ipProtocol                        1000000 non-null  int64         
 3   protocol                          1000000 non-null  string        
 4   rootId                            6031 non-null     string        
 5   totDataBytes                      1000000 non-null  int64         
 6   client.bytes                      1000000 non-null  int64         
 7   server.bytes                      1000000 non-null  int64         
 8   source.ip                         1000000 non-null  object        
 9   source.port                       1000000 non-null  int64         
 10  destination.ip   

None

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000000 entries, 1 to 19433495
Data columns (total 19 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   is_multicast      999315 non-null  object
 1   is_private        999315 non-null  object
 2   is_global         999315 non-null  object
 3   is_unspecified    999315 non-null  object
 4   is_reserved       999315 non-null  object
 5   is_loopback       999315 non-null  object
 6   is_link_local     999315 non-null  object
 7   blue_team         127885 non-null  object
 8   facility          681958 non-null  object
 9   service           999315 non-null  object
 10  critservice       999315 non-null  object
 11  hostname          306029 non-null  object
 12  dns_alias         235391 non-null  object
 13  hardware          306029 non-null  object
 14  opsys             310198 non-null  object
 15  sharedinfra       6400 non-null    object
 16  is_red_team       999315 non-null  

None

# Feature Engineering

In [5]:
CATEGORICAL_COLS = [
    # 'rootId',
    'ipProtocol',
    'protocol',
    'source.ip',
    'source.port',
    'destination.ip',
    'destination.port',
    # 'node',
    # 'http.uri',
    'source.geo.country_iso_code',
    'destination.geo.country_iso_code',
    # 'dns.host',
    'has_dns_host',
    # 'cert',
    # 'irc.channel',
    # 'http.xffGEO',
    'has_cert',
]

COUNTRY_ISO_CODE_COLS = ['source.geo.country_iso_code', 'destination.geo.country_iso_code']

# array_str_cols = ['protocol', 'dns.host', 'service', 'critservice', 'dns_lookup_alias']

NUMERIC_COLS = [
    'totDataBytes',
    'client.bytes',
    'server.bytes',
    'network.packets',
    'source.packets',
    'destination.packets',
    'network.bytes',
    'source.bytes',
    'destination.bytes',
]

LABEL_BINARY_COLS = [
    # 'is_multicast',
    'is_private','is_global',
    'is_unspecified',
    # 'is_reserved',
    # 'is_loopback', # only 6 positive
    'is_link_local',
    'is_red_team',

]
LABEL_MULTI_CLASSES_COLS =[
    'blue_team','facility','service',
    # 'critservice',
    'hostname',
    'dns_alias',
    'hardware',
    'opsys',
    # 'sharedinfra',
    # 'dns_lookup_host',
    # 'dns_lookup_alias'
]

# cyclic_cols = [
#     'hour',
#     'minute',
#     'second',
#     'microsecond'
# ]


new_cyclic_cols = [
    c for col in CYCLIC_COLS for c in [f'{col}_sin', f'{col}_cos']
]

NUMERIC_COLS += new_cyclic_cols

for col in CATEGORICAL_COLS:
    data[col] = data[col].astype('category')


# Train/Test Split

In [6]:
from typing import Union, List, Callable
from sklearn.model_selection import train_test_split
from sklearn.utils.validation import check_is_fitted
from sklearn.preprocessing import LabelEncoder, StandardScaler
from united.data.encoders import NaNLabelEncoder
from united.data.imputers import BackwardFillingImputer, ForwardFillingImputer, InterpolateImputer, TryHardImputer, ConstantFillingImputer
import warnings
from copy import deepcopy

RANDOM_STATE = 42

def split_by_time(
        data: pd.DataFrame, label: pd.DataFrame,
        validation_cutoff = pd.to_datetime('2016-04-22 20:30:00'),
        testing_cutoff = pd.to_datetime('2016-04-22 21:00:00')
    ):
    mask_train = data['lastPacket'] > testing_cutoff
    X_test = data[mask_train]
    y_test = label[mask_train]
    
    mask_val = (data['lastPacket'] > validation_cutoff) & (data['lastPacket'] < testing_cutoff)
    X_val = data[mask_val]
    y_val = label[mask_val]

    mask_train = data['lastPacket'] <= validation_cutoff
    X_train = data[mask_train]
    y_train = label[mask_train]

    return (X_train, y_train), (X_val, y_val), (X_test, y_test)

def split_by_row(data: pd.DataFrame, label: Union[pd.DataFrame, pd.Series], test_size=0.2, val_size=0.1):
    X_train_val, X_test, y_train_val, y_test = train_test_split(data, label, test_size=test_size, random_state=RANDOM_STATE)
    X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=val_size)
    return (X_train, y_train), (X_val, y_val), (X_test, y_test)


# -----------------------------
# SPLIT DATA

def is_numeric(y: pd.Series) -> bool:
    """
    Determine if series is numeric or not. Will also return True if series is a categorical type with
    underlying integers.

    Args:
        y (pd.Series): series for which to carry out assessment

    Returns:
        bool: True if series is numeric
    """
    return y.dtype.kind in "bcif" or (isinstance(y, pd.CategoricalDtype) and y.cat.categories.dtype.kind in "bcif")


def acquire_train_val_test_splits(
    data: pd.DataFrame, 
    label: Union[pd.DataFrame, pd.Series],
    target: str,
    features: List[str], 
    lagged_features: List[str] = [],
    trailing_window_size: int = 0,
    split_fn: Callable = split_by_time,
):
    # xxx
    target_mask = label[target].notna()
    y = label[target][target_mask]
    X = data[target_mask]

    # adding lagged features:
    groups = [
        'destination.ip', 
        'destination.port'
    ]
    add_lagged_features = len(lagged_features) and trailing_window_size > 0 and all([ g in list(X.columns) for g in groups])
    # ic(add_lagged_features)
    lag_mapping = {}
    if add_lagged_features:
        lagged_features = deepcopy(lagged_features)
        assert all([ f in features for f in lagged_features])
        g = X.groupby(by=groups)
        lags = {}
        for feature in lagged_features:
            for window in range(1, trailing_window_size+1):
                shifted = g[feature].shift(window)
                # lagged_name = f"lagged_{feature}_{window}"
                lagged_name = f"{feature}__lagged__{window}"
                lags[lagged_name] = shifted
                lag_mapping[lagged_name] = feature
        lags_df = pd.DataFrame(lags)
        
        X = pd.concat([X, lags_df], axis=1)
        # assign column type (numeric or categorical) for extra lagged features
        for col in list(lags_df.columns):
            org_col = lag_mapping[col]
            if org_col in CATEGORICAL_COLS:
                X[col] = X[col].astype("category")
    
    target_encoder = LabelEncoder().fit(y)
    y = target_encoder.transform(y)
    
    (X_train, y_train), (X_val, y_val), (X_test, y_test) = split_fn(X, y)

    if add_lagged_features:
        features = features + list(lags_df.columns)

    X_train = X_train[features]
    X_test = X_test[features]
    X_val = X_val[features]
    return (X_train, y_train), (X_val, y_val), (X_test, y_test), target_encoder

def feature_to_lag_name(feature, window):
    return f"{feature}__lagged__{window}"
import re
def lag_name_to_feature_window(lag_name):
    parts = re.split('__lagged__', lag_name)[:2]
    return (parts[0], int(parts[1]))


# Training

In [7]:
import xgboost
from sklearn import metrics
from copy import deepcopy

def train_xgboost(X_train, y_train, X_val, y_val, verbose=False):

    xgb_params = {
        "tree_method": "hist",
        # "n_estimators": 32,
        # "colsample_bylevel": 0.7,
    }

    predictor = xgboost.XGBClassifier(
        **xgb_params,
        eval_metric='auc',
        enable_categorical=True,
        max_cat_to_onehot=1,
    )
    predictor.fit(
        X_train, y_train, 
        eval_set=[
            (X_val, y_val), 
            # (X_train, y_train)
        ],
        verbose=verbose
    )
    return predictor


def eval_binary_model(predictor, X_test, y_test, target_encoder: LabelEncoder, debug=False):
    y_proba_preds = predictor.predict_proba(X_test)[:, 1]
    y_preds = y_proba_preds > 0.5
    auc = metrics.roc_auc_score(y_test, y_proba_preds)
    accuracy = metrics.accuracy_score(y_test, y_preds)
    bal_accuracy = metrics.balanced_accuracy_score(y_test, y_preds)
    fpr, tpr, thresholds = metrics.roc_curve(y_test, y_proba_preds)
    confusion_matrix = metrics.confusion_matrix(y_test, y_preds)
    classification_report = metrics.classification_report(y_test, y_preds, target_names=target_encoder.classes_.astype(str))
        
    if debug:
        print(f'AUC: {auc}')
        print(f'Accuracy: {accuracy}')
        print(f'Balance Accuracy: {bal_accuracy}')
        print('Confusion Matrix: ') 
        print(confusion_matrix)
        print(classification_report)
    return auc, accuracy, bal_accuracy, confusion_matrix, classification_report

def plot_roc_curve(predictor, X_test, y_test):
    y_proba_preds = predictor.predict_proba(X_test)[:, 1]
    auc = metrics.roc_auc_score(y_test, y_proba_preds)
    fpr, tpr, thresholds = metrics.roc_curve(y_test, y_proba_preds)
    #create ROC curve
    plt.plot(fpr,tpr)
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.title(f'target_col - AUC={auc}')
    plt.show()
    

def eval_multi_class_model(predictor, X_test, y_test, target_encoder: LabelEncoder, debug=False):
    multi_class = len(predictor.classes_) > 2
    y_proba_preds = predictor.predict_proba(X_test)
    if multi_class:
        y_preds = predictor.predict(X_test)
    else:
        y_proba_preds = y_proba_preds[:, 1]
        y_preds = y_proba_preds > 0.5
    auc = metrics.roc_auc_score(y_test, y_proba_preds, multi_class='ovo' if multi_class else 'raise')
    accuracy = metrics.accuracy_score(y_test, y_preds)
    bal_accuracy = metrics.balanced_accuracy_score(y_test, y_preds)
    confusion_matrix = metrics.confusion_matrix(y_test, y_preds)
    classification_report = metrics.classification_report(y_test, y_preds, target_names=target_encoder.classes_.astype(str))
    
    if debug:
        print(f'AUC: {auc}')
        print(f'Accuracy: {accuracy}')
        print(f'Balance Accuracy: {bal_accuracy}')
        print('Confusion Matrix: ')
        print(confusion_matrix)
        # metrics.ConfusionMatrixDisplay()
        print("Classification Report:")
        print(classification_report)
    return auc, accuracy, bal_accuracy, confusion_matrix, classification_report


def get_feature_importance(predictor, X_train, topk=5, debug=False):
    d = []
    for val, name in zip(predictor.feature_importances_, X_train.columns):
        d.append((name, val))
    # display(d)
    sorted_d = sorted(d, key=lambda item: item[1], reverse=True)
    if debug:
        print(f"Feature important - Top {topk} Features:")
        topk = min(topk, len(sorted_d))
        print(sorted_d[:topk])
    return sorted_d



def run_full_pipeline(
    target_col, split_fn, 
    features=NUMERIC_COLS+CATEGORICAL_COLS, 
    lagged_features=NUMERIC_COLS, 
    drop_source_ip=False, 
    drop_destination_ip=False, 
    trailing_window=0, 
    train_verbose=False, 
    print_result=True
):
    # features = numeric_cols + categorical_cols
    # lagged_features = numeric_cols
    # ic(features)
    # ic(lagged_features)
    to_remove_features = []
    if drop_source_ip:
        to_remove_features += ['source.ip', 'source.port']
    if drop_destination_ip:
        to_remove_features += ['destination.ip', 'destination.port']
    
    for f in to_remove_features:
        try:
            features.remove(f)
        except:
            pass
        try:
            lagged_features.remove(f)
        except:
            pass
    

    (X_train, y_train), (X_val, y_val), (X_test, y_test), target_encoder = acquire_train_val_test_splits(
        data, label, 
        target=target_col,
        features=features,
        lagged_features=lagged_features,
        trailing_window_size=trailing_window,
        split_fn=split_fn
    )
    print('Features: ', list(X_train.columns))
    print('Target Name: ', target_col)
    print(f'Train size: {X_train.shape}')
    print(f'Val size: {X_val.shape}')
    print(f'Test size: {X_test.shape}')
    print(f"Total classes: {len(target_encoder.classes_)}, Classes: {target_encoder.classes_[:min(10, len(target_encoder.classes_))]}")

    predictor = train_xgboost(X_train, y_train, X_val, y_val, verbose=train_verbose)
    
    eval_fn = eval_multi_class_model
    auc, accuracy, bal_accuracy, confusion_matrix, classification_report = eval_fn(predictor, X_test, y_test, target_encoder, debug=print_result)
    feature_importances = get_feature_importance(predictor, X_train, debug=print_result, topk=10)
    return auc, accuracy, bal_accuracy, confusion_matrix, classification_report, feature_importances


# Generate report

In [8]:

import pickle
from datetime import datetime
from copy import deepcopy

FEATURES = NUMERIC_COLS + CATEGORICAL_COLS
LAGGED_FEATURES = NUMERIC_COLS + CATEGORICAL_COLS

# drop_source_ips = [False, True]
# drop_destination_ips = [False, True]
# split_fns = [split_by_time, split_by_row]
# target_cols = label_binary_cols + label_multi_classes_cols
# trailing_window_sizes = [0, 1, 5, 10, 20]

drop_source_ips = [True]
drop_destination_ips = [True]
split_fns = [split_by_time]
target_cols =  LABEL_MULTI_CLASSES_COLS
trailing_window_sizes = [0, 1, 10, 20]


# drop_source_ips = [True]
# drop_destination_ips = [True]
# split_fns = [split_by_time]
# target_cols = label_binary_cols + label_multi_classes_cols; target_cols = target_cols[:1]
# trailing_window_sizes = [0, 5]

# Make sure data and label are sorted: combine => sort => split
combined = pd.concat([data, label], axis=1).sort_values(by='time_idx')
data = combined[list(data.columns)]
label = combined[list(label.columns)]

trials = []

for target_col in target_cols:
    for drop_source_ip in drop_source_ips:
        for drop_destination_ip in drop_destination_ips:
            for split_fn in split_fns:
                for trailing_w in trailing_window_sizes:
                    print('+'*120)
                    print(f'Classification Report for {target_col} - Split_fn = {split_fn.__name__} - Drop source_ip/port = {drop_source_ip} - Drop destination ip/port = {drop_destination_ip} - Window = {trailing_w}')
                    print('+'*120)
                    try:
                        # if trailing_w > 0 and drop_destination_ip:
                        #     continue
                        auc, accuracy, bal_accuracy, confusion_matrix, classification_report, feature_importances = run_full_pipeline(
                            target_col, split_fn=split_fn,
                            features=deepcopy(FEATURES),
                            lagged_features=deepcopy(LAGGED_FEATURES),
                            drop_source_ip=drop_source_ip,
                            drop_destination_ip=drop_destination_ip, 
                            trailing_window=trailing_w,
                            train_verbose=False, print_result=True
                        )
                        trail = {
                            'target': target_col,
                            'split_fn': split_fn.__name__,
                            'drop_source_ip': drop_source_ip,
                            'drop_destination_ip': drop_destination_ip,
                            'window': trailing_w,
                            'auc': auc,
                            'accuracy': accuracy,
                            'bal_accuracy': bal_accuracy,
                            'confusion_matrix': confusion_matrix,
                            'classification_report': classification_report,
                            'feature_importances': feature_importances,
                        }
                        trials.append(trail)
                    except ValueError as e:
                        # import traceback
                        # print(traceback.print_exc())
                        # raise e
                        print(e)

# save experiment results
results = pd.DataFrame(trials)
now = datetime.now()
display(results)
with open(f'reports/data/trial_results_n_trails_{len(trials)}_{hostname}_{now.strftime("%Y_%m_%d_%H_%M_%S")}.pkl', 'wb') as fp:
    pickle.dump(results, fp)
            

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Classification Report for blue_team - Split_fn = split_by_time - Drop source_ip/port = True - Drop destination ip/port = True - Window = 0
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Features:  ['totDataBytes', 'client.bytes', 'server.bytes', 'network.packets', 'source.packets', 'destination.packets', 'network.bytes', 'source.bytes', 'destination.bytes', 'hour_sin', 'hour_cos', 'minute_sin', 'minute_cos', 'second_sin', 'second_cos', 'microsecond_sin', 'microsecond_cos', 'ipProtocol', 'protocol', 'source.geo.country_iso_code', 'destination.geo.country_iso_code', 'has_dns_host', 'has_cert']
Target Name:  blue_team
Train size: (61255, 23)
Val size: (6007, 23)
Test size: (60623, 23)
Total classes: 10, Classes: ['Team 1' 'Team 10' 'Team 2' 'Team 3' 'Team 4' 'Team 5' 'Team 6' 'Team 7'
 'Team 8' 'T

# Hyper Tunning by Optuna

In [ ]:
# import optuna

# drop_source_ips = [False, True]
# split_fns = ['split_by_time', 'split_by_row']
# target_cols = label_binary_cols + label_mulclass_cols

# def objective(trial: optuna.trial.Trial):
#     target_col = trial.suggest_categorical('target', target_cols)
#     split_fn = trial.suggest_categorical('train_test_split_fn', split_fns)
#     split_fn = globals()[split_fn]
#     drop_source_ip = trial.suggest_categorical('drop_source_ip_port', drop_source_ips)
#     try:
#         auc, accuracy, bal_accuracy, confusion_matrix, classification_report, feature_importances = run_full_pipeline(target_col, split_fn=split_fn, drop_source_ip=drop_source_ip, train_verbose=False, print_result=True)
#         trial.set_user_attr('AUC', auc)
#         trial.set_user_attr('ACCURACY', accuracy)
#         trial.set_user_attr("BAL_ACC", bal_accuracy)
#         trial.set_user_attr("CONFUSION_MATRIX", str(confusion_matrix))
#         trial.set_user_attr("CLASSIFICATION_REPORT", classification_report)
#         trial.set_user_attr("FEATURE_IMPORTANCES", feature_importances)
#     except Exception as e:
#         print(e)
#         trial.set_user_attr("EXCEPTION", str(e))
#         return 0
#     return auc

# study = optuna.create_study(storage="sqlite:///db/ncdc_xgboost.sqlite3", study_name='ncdc_xgboost', load_if_exists=True, direction='maximize')
# study.optimize(objective, n_trials=None)


In [ ]:
# # fixing results
# log_file = 'reports/ncdc_xgboost_full_add_lags.log'
# with open('reports/data/trial_results_n_trails_240_cpu-08_2023_04_22_21_19_51.pkl', 'rb') as fp:
#     df = pickle.load(fp)

# drop_source_ips = [False, True]
# drop_destination_ips = [False, True]
# split_fns = ['split_by_time', 'split_by_row']
# target_cols = label_binary_cols + label_multi_classes_cols
# trailing_window_sizes = [0, 1, 5, 10]


# template = f'Classification Report for {target_col} - Split_fn = {split_fn} - Drop source_ip/port = {drop_source_ip} - Drop destination ip/port = {drop_destination_ip} - Window = {trailing_w}'

# trials = []
# df['window'] = 0
# with open(log_file, 'r') as fp:
#     lines = fp.read().splitlines()
#     for target_col in target_cols:
#         for drop_source_ip in drop_source_ips:
#             for drop_destination_ip in drop_destination_ips:
#                 for split_fn in split_fns:
#                     windows = []
#                     for trailing_w in trailing_window_sizes:
#                         if trailing_w > 0 and drop_destination_ip:
#                             continue
#                         template = f'Classification Report for {target_col} - Split_fn = {split_fn} - Drop source_ip/port = {drop_source_ip} - Drop destination ip/port = {drop_destination_ip} - Window = {trailing_w}'
#                         for line in lines:
#                             if line == template:
#                                 # ic(trailing_w, line.removesuffix('\n'))
#                                 windows.append(trailing_w)
#                     if windows:
#                         # ic(windows)
#                         mask = (df['target'] == target_col) & (df['split_fn']==split_fn) & (df['drop_source_ip'] == drop_source_ip) & (df['drop_destination_ip'] == drop_destination_ip)
#                         # ic(df[mask]['window'].shape, len(windows))
#                         df.loc[mask, ['window']] = windows
        
# import pickle
# pickle.dump(df, open('reports/data/trial_results_n_trails_240_cpu-08_2023_04_22_21_19_51_fixed_windows.pkl', 'wb'))

# Analysize Results

In [ ]:
import pickle
# with open('reports/data/trial_results_n_trails_240_cpu-08_2023_04_22_21_19_51_fixed_windows.pkl', 'rb') as fp:
# with open('reports/data/trial_results_n_trails_240_cpu-08_2023_04_22_21_19_51.pkl', 'rb') as fp:
# with open('reports/data/trial_results_n_trails_28_cpu-02_2023_04_26_21_12_25.pkl', 'rb') as fp:
with open('reports/data/trial_results_n_trails_28_NVD4K9M6K4_2023_05_02_12_48_53.pkl', 'rb') as fp:
    df = pickle.load(fp)

str_cols = ['confusion_matrix', 'classification_report', 'feature_importances']
for col in str_cols:
    df[col] = df[col].astype(str)

float_cols = ['auc', 'bal_accuracy', 'accuracy']
for col in float_cols:
    df[col] = df[col].apply(lambda x: round(x, 2))

df

,target,split_fn,drop_source_ip,drop_destination_ip,window,auc,accuracy,bal_accuracy,confusion_matrix,classification_report,feature_importances
0,blue_team,split_by_time,True,True,0,0.76,0.49,0.32,[[ 66 26 63 73 100 60 116 ...,precision recall f1-score ...,"[('protocol', 0.12349972), ('network.bytes', 0..."
1,blue_team,split_by_time,True,True,1,0.79,0.54,0.35,[[ 56 11 85 72 147 90 111 ...,precision recall f1-score ...,"[('protocol__lagged__1', 0.11651987), ('hour_c..."
2,blue_team,split_by_time,True,True,10,0.80,0.57,0.36,[[ 82 28 78 62 149 95 116 ...,precision recall f1-score ...,"[('protocol__lagged__10', 0.037768282), ('dest..."
3,blue_team,split_by_time,True,True,20,0.80,0.59,0.37,[[ 59 17 97 59 162 123 86 ...,precision recall f1-score ...,"[('protocol__lagged__16', 0.050368007), ('dest..."
4,facility,split_by_time,True,True,0,0.97,0.90,0.69,[[ 706 70 11]\n [ 1 155149 4...,precision recall...,"[('source.bytes', 0.12454854), ('destination.b..."
5,facility,split_by_time,True,True,1,0.97,0.91,0.73,[[ 730 55 2]\n [ 3 155130 5...,precision recall...,"[('server.bytes__lagged__1', 0.18684399), ('ho..."
6,facility,split_by_time,True,True,10,0.94,0.91,0.72,[[ 728 58 1]\n [ 2 154960 23...,precision recall...,"[('server.bytes__lagged__2', 0.07993748), ('se..."
7,facility,split_by_time,True,True,20,0.93,0.91,0.71,[[ 743 43 1]\n [ 3 154955 23...,precision recall...,"[('server.bytes__lagged__9', 0.09269878), ('se..."
8,service,split_by_time,True,True,0,0.86,0.75,0.31,[[ 486 5 1 0 0 1 ...,precision recall f1-scor...,"[('hour_sin', 0.3071031), ('source.bytes', 0.1..."
9,service,split_by_time,True,True,1,0.87,0.75,0.32,[[ 31 2 2 0 0 4 ...,precision recall f1-scor...,"[('hour_sin', 0.26280984), ('totDataBytes__lag..."


In [ ]:
import plotly.express as px

# fig = px.parallel_coordinates(
objective = 'auc'
# objective = 'bal_accuracy'

fig = px.parallel_categories(
    df,
    # df[df['drop_destination_ip']==False],
    # df[
    #     (df['target'].isin({'service', 'hostname', 'hardware', 'opsys'})) 
    #     & (df['drop_destination_ip']==False)
    #     & (df['drop_source_ip']==True)
    #     & (df['split_fn'] == 'split_by_time')
    # ],
    color=objective,
    dimensions=[
        'target', 
        # 'split_fn', 
        # 'drop_source_ip',
        # 'drop_destination_ip',
        'window',
        # objective,
    ],
    color_continuous_scale=px.colors.diverging.Tealrose,
    # color_continuous_midpoint=0.5,
    # dimensions_max_cardinality=5,

)
fig.show()

In [ ]:
# fig.write_html('reports/parallel_categories_iternative_chart_multi_classes_target_vs_windows.html')